In [21]:
adjacency_list = [
    [1,2,3],
    [0,2,3],
    [0,1,3],
    [0,1,2],
    [0,3]
]

edge_list = [
    
]

adjacency_matrix = [
    [0,1,1,1,0],
    [1,0,1,1,0],
    [1,1,0,1,0],
    [1,1,1,0,0],
    [1,0,0,1,0],
]

def adj_list_to_mat( adj_list ):
    
    return [ [1 if i in row else 0 for i, _ in enumerate(adj_list) ] 
            for row in adj_list ]
    

def adj_list_to_edge_list( adj_list ):
    
    return [ (i,j) for i, E in enumerate(adj_list) for j in E ]


def adj_mat_to_list( adj_mat ):
    
    return [ [i for i, v in enumerate(row) if v] 
            for row in adj_mat ]


In [22]:
adj_list_to_edge_list( adjacency_list )

[(0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 3)]

In [40]:
G1_adjl = [
    [1, 2], # 0
    [2, 3], # 1
    [3, 4], # 2
    [5], # 3
    [5], # 4
    [], # 5
]

In [36]:
class Queue:
    def __init__(self):
        self.list = []
        self.set = set()
        self.front = 0
        
    def push( self, x ):
        self.list.append(x)
        self.set.add(x)
    
    def pop( self ):
        if self.empty():
            return None
        self.front += 1
        return self.list[self.front - 1]
    
    def empty( self ):
        return self.front == len(self.list)
    
    def has_contained( self, x ):
        return x in self.set

In [38]:
def bfs( G_adjl, s ):
    
    queue = Queue()
    
    queue.push(s)
    
    while not queue.empty():
        
        current = queue.pop()
        print(current)
        
        for connection in G_adjl[current]:
            if queue.has_contained(connection):
                continue
            queue.push(connection)

In [43]:
bfs(G1_adjl, 4)

4
5


In [24]:
G_adjl = [
    [(1,6),(2,1),(3,1)],
    [(0,1),(2,1),(3,1)],
    [(0,1),(1,1),(3,1)],
    [(0,1),(1,1),(2,1)]
]


def mst( G_adjl, source_index=0 ):
    
    N = len( G_adjl )
    frontier = [ source_index ]
    
    spanning_tree = []
    
    
    while True:
        
        candidates = [
            (i,j,w) for i, connections in enumerate([G_adjl[i] for i in frontier])
            for (j,w) in connections
            if j not in frontier and i != j
        ]
        
        if not candidates:
            return spanning_tree
        
        print(candidates)
        
        min_edge = min( candidates, key=lambda e: e[2] )
        
        spanning_tree.append( min_edge )
        frontier.append( min_edge[1] )

In [25]:
mst( G_adjl )

[(0, 1, 6), (0, 2, 1), (0, 3, 1)]
[(0, 1, 6), (0, 3, 1), (1, 3, 1)]
[(0, 1, 6), (2, 1, 1)]


[(0, 2, 1), (0, 3, 1), (2, 1, 1)]